In [1]:
import pandas as pd
import numpy as np
# import matplotlib.pyplot as plt

In [2]:
path = r'./data/raw/'
name = 'data_2024-05-03_15-32.csv'

data_raw = pd.read_csv(path+name)

# Data Cleaning

In [3]:
n_errors = data_raw[data_raw['Name']=='error'].shape[0]
perc = n_errors / data_raw.shape[0]
print(f'{n_errors} car ads of {data_raw.shape[0]} does not scraped. Percentage: {round(perc*100,2)}%')

data_clean = data_raw[data_raw['Name']!='error'].reset_index(drop=True)

6843 car ads of 25895 does not scraped. Percentage: 26.43%


**Clean the brand name column**

In [4]:
data_clean['Name_clean'] = data_clean['Name'].apply(lambda text: text.split()[0])
data_clean = data_clean[data_clean['Name_clean']!='Αυτοκίνητο']
conditions = [data_clean['Name_clean']=='Mini', data_clean['Name_clean']=='Land', data_clean['Name_clean']=='Alfa', data_clean['Name_clean']=='Aston', data_clean['Name_clean']=='Rolls', data_clean['Name_clean']=='Austin']
values = ['Mini-Cooper', 'Land-Rover', 'Alfa-Romeo', 'Aston-Martin', 'Rolls-Royce', 'Austin-Morris']
data_clean['Name_clean'] = np.select(conditions, values, default=data_clean['Name_clean'])

**Split and clean "Info" column to individuals and merge to the main dataframe**

In [5]:
info_columns = data_clean['Info'].str.split(',', expand=True)
info_columns[0] = pd.to_datetime(info_columns[0], format='%m/%Y')

info_columns = info_columns[info_columns[1].str.contains('χλμ')]
info_columns[1] = info_columns[1].str.replace('.', '',regex=False).str.replace('χλμ', '',regex=False).astype(float)

info_columns = info_columns[info_columns[2].str.contains('cc')]
info_columns[2] = info_columns[2].str.replace('.', '',regex=False).str.replace('cc', '',regex=False).astype(float)

info_columns[3] = info_columns[3].str.replace('.', '',regex=False).str.replace('bhp', '',regex=False).astype(float)

conditions = [info_columns[4].isin([' Αυτόματο', ' Ημιαυτόματο'])]
values = [info_columns[4]]
info_columns['GearBox'] = np.select(conditions, values, default='Manual')
info_columns['GearBox'] = info_columns['GearBox'].str.strip(' ')

conditions = [(info_columns[4] == ' Αυτόματο') | (info_columns[4] == ' Ημιαυτόματο')]
values = [info_columns[5]]
info_columns[4] = np.select(conditions, values, default=info_columns[4])
info_columns[4] = info_columns[4].str.strip(' ')

info_columns.drop([5,6,7], axis=1, inplace=True)

data_clean_merged = pd.merge(data_clean, info_columns, left_index=True, right_index=True)
data_clean_merged.drop(['Info'], axis=1, inplace=True)

data_clean_merged.columns = ['FullName','Price','Name','ProductionDate','Klm','CubicCapacity','Horsepower','GasType', 'GearBox']

# Feature Engineering

In [6]:
reference_year = 2024
data_clean_merged['Age'] = reference_year - data_clean_merged['ProductionDate'].dt.year
data_clean_merged.drop(['FullName', 'ProductionDate'], inplace=True, axis=1)

**Outliers Halding**

In [7]:
data_clean_merged_filtered = data_clean_merged[data_clean_merged['Price']>1000]

**Filter out the car brands that have less than 20 ads. Not big reduction of the dataset**

In [8]:
counts_df = pd.DataFrame(data_clean_merged_filtered['Name'].value_counts())
counts_df.columns = ['count']
counts_df = counts_df[counts_df['count'] >= 20]
name_list = list(counts_df.index)

print(f'{data_clean_merged_filtered.shape[0]} rows, {data_clean_merged_filtered.shape[1]} attributes before filter')
data_clean_merged_filtered = data_clean_merged_filtered[data_clean_merged_filtered['Name'].isin(name_list)]
print(f'{data_clean_merged_filtered.shape[0]} rows, {data_clean_merged_filtered.shape[1]} attributes after filter')

18481 rows, 8 attributes before filter
18304 rows, 8 attributes after filter


# Save Data

In [9]:
def save_data(save_path, file_name):
    data_clean_merged_filtered.to_csv(save_path+file_name, sep=';', index=False)
    print('Data Saved')
    
save_path = r'./data/clean/'
save_data(save_path, 'data_clean_20240509.csv')

Data Saved
